In [1]:
import nltk
nltk.download('punkt')
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

[nltk_data] Downloading package punkt to /home/mohammad/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
import numpy
import random 
import tensorflow as tf
import json 
#impoort the split 
from sklearn.model_selection import train_test_split
import numpy as np
#import keras classifier   
#IMPORY SGD 
from tensorflow.keras.optimizers import SGD


In [3]:

with open("/home/mohammad/Desktop/First_AId-1.0-sequence-to-sequence-/intents (2).json") as file:
    data = json.load(file)   


In [4]:
import numpy as np

# Define preprocessing functions
def tokenize(sentence):
    return nltk.word_tokenize(sentence)

def stem(word):
    return stemmer.stem(word.lower())

# Initialize lists for words, labels, training data, and output
words = []
labels = []
training_data = []
output_data = []

# Tokenize and stem each pattern, and prepare training data and output
for intent in data["intents"]:
    for pattern in intent["patterns"]:
        tokens = tokenize(pattern)
        words.extend(tokens)
        training_data.append(tokens)
        output_data.append(intent["tag"])

        if intent["tag"] not in labels:
            labels.append(intent["tag"])

# Stem and lowercase words, remove duplicates, and sort
words = sorted(list(set([stem(word) for word in words if word != "?"])))
labels = sorted(labels)

# Initialize empty output rows
out_empty = [0] * len(labels)

# Prepare training and output arrays
X = []
y = []

for idx, doc in enumerate(training_data):
    bag = []

    # Stem and lowercase words in the pattern
    tokens = [stem(word) for word in doc]

    # Create bag of words
    for word in words:
        bag.append(1) if word in tokens else bag.append(0)

    # Create output row
    output_row = list(out_empty)
    output_row[labels.index(output_data[idx])] = 1

    X.append(bag)
    y.append(output_row)

# Conve
training = np.array(X)
output = np.array(y)




In [5]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

# Reset the default graph
tf.reset_default_graph()

# Define the model
model = tf.keras.Sequential([
    # Input layer (implicit by specifying input_shape in the first Dense layer)
    tf.keras.layers.Dense(128, activation='relu', input_shape=(len(training[0]),)),  
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(64, activation='relu'),  
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(len(output[0]), activation='softmax')
])

# Define optimizer
sgd = tf.train.MomentumOptimizer(learning_rate=0.01, momentum=0.9, use_nesterov=True)

model.compile(optimizer=sgd,
              loss='categorical_crossentropy',
              metrics=['accuracy']
              )    

# Fit the model
model.fit(training, output, epochs=350, batch_size=5, verbose=1)


Instructions for updating:
non-resource variables are not supported in the long term
Train on 995 samples
Epoch 1/350
995/995 [==============================] - 0s 223us/sample - loss: 5.9408 - acc: 0.0201
Epoch 2/350
995/995 [==============================] - 0s 124us/sample - loss: 5.3570 - acc: 0.0412
Epoch 3/350
995/995 [==============================] - 0s 122us/sample - loss: 4.8733 - acc: 0.1045
Epoch 4/350
995/995 [==============================] - 0s 128us/sample - loss: 4.4673 - acc: 0.1688
Epoch 5/350
995/995 [==============================] - 0s 132us/sample - loss: 4.1260 - acc: 0.2312
Epoch 6/350
995/995 [==============================] - 0s 126us/sample - loss: 3.8463 - acc: 0.2593
Epoch 7/350
995/995 [==============================] - 0s 121us/sample - loss: 3.5323 - acc: 0.3015
Epoch 8/350
995/995 [==============================] - 0s 118us/sample - loss: 3.2578 - acc: 0.3407
Epoch 9/350
995/995 [==============================] - 0s 120us/sample - loss: 3.0052 - acc: 0

In [6]:
model 

In [7]:
def bag_of_words(s,words):
	bag = [0 for _ in range(len(words))]


	s_words = nltk.word_tokenize(s)
	s_words = [stemmer.stem(word.lower()) for word in s_words]

	for se in s_words:
		for i, w in enumerate(words):
			if w == se:
				bag[i] = 1

	return numpy.array(bag)

In [8]:
def chat():
    print("Start Talking with the bot (type quit to stop)")
    while True:
        inp = input("You: ")
        if inp.lower() == "quit":
            break

        # Preprocess input text into bag of words
        inp_bag_of_words = bag_of_words(inp, words)

        # Reshape input for model prediction
        inp_bag_of_words = inp_bag_of_words.reshape(1, inp_bag_of_words.shape[0])

        # Predict using the model
        results = model.predict(inp_bag_of_words)[0]

        results_index = numpy.argmax(results)
        tag = labels[results_index]

        if results[results_index] > 0.3:
            for tg in data["intents"]:
                if tg['tag'] == tag:
                    responses = tg['responses']
            print(random.choice(responses))
            print("\n")

        else:
            print("I didn't get that, try again")
